In [ ]:
from dask.distributed import Client, progress
import dask.array as da
import numpy as np
import scipy
client = Client(processes=False)
client

Overlapping Operations
======================

Some operations depend on neigbourding values. For exemple derivatives, sliding sum, image filter.

For that there is `overlap` and `map_overlap`.  
It add a border to each chunks before mapping function on each chunks.  

![](https://docs.dask.org/en/stable/_images/overlapping-neighbors.svg)

It does so in all dimension, including diagonals:

![](https://docs.dask.org/en/stable/_images/overlapping-blocks.svg)

In [ ]:
x = da.from_array(np.arange(100).reshape((10, 10)), chunks=(5, 2))
x

In [ ]:
x.compute()

In [ ]:
extented = da.overlap.overlap(x, depth=(1, 1), boundary=("reflect", -100)) 
extented

In [ ]:
extented.compute()[:, :10]

Outer border can be:
- "periodic"
- "reflect"
- Any constant

Once extented, we can map a function on each blocks.  
This is good for scipy array functions.


In [ ]:
def convolve2d(arr):
    filter = np.array([[0, 1, 0],[1, -4, 1],[0, 1, 0]])
    return scipy.signal.convolve2d(arr, filter, mode="same")

filtered = extented.map_blocks(convolve2d)
filtered

In [ ]:
filtered.blocks[0, 0].compute()

In [ ]:
filtered.blocks[0, 1].compute()

Often once the action on each block is done, we often want to remove the borders.  
This can be done with `trim_overlap` or `trim_internal`.
- `trim_overlap`: only trim the overlap between blocks.
- `trim_internal`: Trim overlap between blocks and boundary, (in 2 parameters).

In [ ]:
trimmed = da.overlap.trim_overlap(filtered, (1, 1))
trimmed

In [ ]:
trimmed.blocks[0, 0].compute()

In [ ]:
trimmed2 = da.overlap.trim_internal(filtered, axes={0:1, 1:1}, boundary={0:1, 1:1})
trimmed2

In [ ]:
trimmed2.blocks[0, 0].compute()

In [ ]:
trimmed2.compute()

### Utility function: `map_overlap`

`map_overlap` joint the 2 steps in one: create the overlap, apply the function, trim the overlap back.

In [ ]:
out = da.overlap.map_overlap(convolve2d, x, depth=(1, 1), boundary=("reflect", -100))
out.compute()

## Exercise: Sliding windows

Let's apply scipy's sliding maximum function to dask arrays.

In [ ]:
# Import and example

from scipy.ndimage import maximum_filter1d, minimum_filter1d
from scipy.ndimage import maximum_filter, minimum_filter
import matplotlib.pyplot as plt
N = 10
signal = np.random.rand(N)
max_1d = maximum_filter1d(signal, 3)
min_1d = minimum_filter1d(signal, 3)
plt.plot(signal)
plt.plot(max_1d)
plt.plot(min_1d)

In [ ]:
N = 100000
signal = np.random.rand(N)
max_1d = maximum_filter1d(signal, 5)
min_1d = minimum_filter1d(signal, 5)

In [ ]:
N = 100
noise = np.random.rand(N, N)
max_2d = maximum_filter(noise, 5)
min_2d = minimum_filter(noise, 5)

### Solution
<!---
N = 100000
signal = da.random.random(N, chunks=(10000))
extended = da.overlap.overlap(signal, (2,), ("reflect",))
max_1d = da.overlap.map_blocks(lambda arr: maximum_filter1d(arr, 5), extended)
min_1d = da.overlap.map_blocks(lambda arr: minimum_filter1d(arr, 5), extended)
max_1d = da.overlap.trim_internal(max_1d, {0: 2,}, {0: 2,})
min_1d = da.overlap.trim_internal(min_1d, {0: 2,}, {0: 2,})

assert np.allclose(max_1d.compute(), maximum_filter1d(signal.compute(), 5))

N = 100
noise = da.random.random((N, N), chunks=(10000))
extended = da.overlap.overlap(noise, (2, 2), ("reflect", "reflect"))
max_2d = da.overlap.map_blocks(lambda arr: maximum_filter(arr, 5), extended)
min_2d = da.overlap.map_blocks(lambda arr: minimum_filter(arr, 5), extended)
max_2d = da.overlap.trim_internal(max_2d, {0: 2, 1: 2,}, {0: 2, 1: 2,})
min_2d = da.overlap.trim_internal(min_2d, {0: 2, 1: 2,}, {0: 2, 1: 2,})

assert np.allclose(max_2d.compute(), maximum_filter(noise.compute(), 5))
--->